# DL4G - Jass Introduction

In this exercise we will look at some properties of the jass kit environment that can be used to develop your own jass agent.

You will need to have numpy installed, as well as the jass-kit environment.

In [2]:
from jass.game.game_util import *
from jass.game.game_sim import GameSim
from jass.game.game_observation import GameObservation
from jass.game.const import *
from jass.game.rule_schieber import RuleSchieber
from jass.agents.agent import Agent
from jass.agents.agent_random_schieber import AgentRandomSchieber
from jass.arena.arena import Arena


Information about the cards is stored as one-hot encoded arrays, there are several tools available to access the information in the cards. 

Lets deal some random cards first.

In [3]:
# Lets set the seed of the random number generater, so that we get the same results
np.random.seed(1)

# This distributes the cards randomly among the 4 players.
hands = deal_random_hand()
print(hands.shape)

(4, 36)


In [4]:
# There is an entry for each player, to access the cards of the first player
cards = hands[0, :]
print(cards)
print("")

[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 0 0 0 0 1 1 1 0 0 0 1 0]



In [5]:
# This should be 9 cards
assert (cards.sum() == 9)

# The cards can be converted to other formats for easier reading or processing
print(convert_one_hot_encoded_cards_to_str_encoded_list(cards))

# Each card is encoded as a value between 0 and 35.
print(convert_one_hot_encoded_cards_to_int_encoded_list(cards))


['DJ', 'H6', 'SK', 'SJ', 'S9', 'CK', 'CQ', 'CJ', 'C7']
[3, 17, 19, 21, 23, 28, 29, 30, 34]


In [6]:
# There is a method to count colors too
colors = count_colors(cards)
print(colors)

[1 1 3 4]


There is a common jass "rule" to select trump, when you have the "Puur" (Jack of trump) and 3 or more other cards of the same color. 

Task 1: Write a function that returns an array of 4 values that contains a 1 for each color that fulfills the rule or 0 otherwise, i.e. [0 0 0 0] is returned, if you do not have any color with Jack and 3 other cards.


In [7]:
def havePuurWithFour(hand: np.ndarray) -> np.ndarray:
    result = np.zeros(4, dtype=int)
    print(hand)
    # add your code here
    colors = count_colors(hand)

    if hand[DJ] == 1 and colors[0] >= 4: result[0] = 1
    if hand[HJ] == 1 and colors[1] >= 4: result[1] = 1
    if hand[SJ] == 1 and colors[2] >= 4: result[2] = 1
    if hand[CJ] == 1 and colors[3] >= 4: result[3] = 1

    return result


havePuurWithFour(cards)

[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 0 0 0 0 1 1 1 0 0 0 1 0]


array([0, 0, 0, 1])

In [8]:
assert (havePuurWithFour(cards) == [0, 0, 0, 1]).all()
cards_2 = hands[1, :]
assert (havePuurWithFour(cards_2) == [0, 0, 0, 0]).all()

[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 0 0 0 0 1 1 1 0 0 0 1 0]
[0 0 1 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 1 0 0]


Another possibility to select trump is by assigning a value to each card, depending on whether the color is trump or not. This table is from the Maturawork of Daniel Graf from 2009: "Jassen auf Basis der Spieltheorie".

In [9]:
# Score for each card of a color from Ace to 6

# score if the color is trump
trump_score = [15, 10, 7, 25, 6, 19, 5, 5, 5]
# score if the color is not trump
no_trump_score = [9, 7, 5, 2, 1, 0, 0, 0, 0]
# score if obenabe is selected (all colors)
obenabe_score = [14, 10, 8, 7, 5, 0, 5, 0, 0, ]
# score if uneufe is selected (all colors)
uneufe_score = [0, 2, 1, 1, 5, 5, 7, 9, 11]

Task 2: Implement a function that evaluates a hand that is given as a list of 9 cards and with a given trump value and returns a score depending on the table above. For example the score of our hand ['DJ', 'H6', 'SK', 'SJ', 'S9', 'CK', 'CQ', 'CJ', 'C7'] when Club is trump should be:

2 + 0 + 7 + 2 + 0 + 10 + 7 + 25 + 5 = 58

while the score is 70 if Spade is selected, which is better as you have both the jack and the nine.

You can use the arrays offset_of_card and color_of_card to get the offset (Ace, King, etc.) and color of a card.

In [10]:
def calculate_trump_selection_score(cards, trump: int) -> int:
    # add your code here
    trump_selection_score = 0
    for card in cards:
        if color_of_card[card] == trump:
            trump_selection_score += trump_score[offset_of_card[card]]
        else:
            trump_selection_score += no_trump_score[offset_of_card[card]]

    return trump_selection_score

In [11]:
card_list = convert_one_hot_encoded_cards_to_int_encoded_list(cards)
assert calculate_trump_selection_score(card_list, CLUBS) == 58
assert calculate_trump_selection_score(card_list, SPADES) == 70

## Agents

In order to play a game you have to program an agent that decides on the action. For that you have to override the methods action_trump and action_play_card.

Task 3: Use the function implemented above to select the best trump value. If the calculated trump value is below a threshold (for example let us take 68, as suggested in the work by Daniel Graf) you should "Schiebe", i.e. pass to your partner if you are still allowed to do that.

The game observation allows you to access the information about your card, and if you are the first or second player to select trump.

For playing a card, we just take a random action.

In [12]:
class MyAgent(Agent):
    def __init__(self):
        super().__init__()
        # we need a rule object to determine the valid cards
        self._rule = RuleSchieber()

    def action_trump(self, obs: GameObservation) -> int:
        """
        Determine trump action for the given observation
        Args:
            obs: the game observation, it must be in a state for trump selection

        Returns:
            selected trump as encoded in jass.game.const or jass.game.const.PUSH
        """
        # add your code here using the function above
        push_threshold = 68
        hand_cards = convert_one_hot_encoded_cards_to_int_encoded_list(obs.hand)
        color_trump_values = [0,0,0,0]
        for color in range(4):
            color_trump_values[color] = calculate_trump_selection_score(hand_cards,color)
        max_value = max(color_trump_values)
        best_color = color_trump_values.index(max_value)
        if max_value < push_threshold and obs.player < 1:
            return PUSH
        else:
            return best_color


    def action_play_card(self, obs: GameObservation) -> int:
        """
        Determine the card to play.

        Args:
            obs: the game observation

        Returns:
            the card to play, int encoded as defined in jass.game.const
        """
        valid_cards = self._rule.get_valid_cards_from_obs(obs)
        # we use the global random number generator here
        return np.random.choice(np.flatnonzero(valid_cards))



We can use the game simulation to play a game. We will use that to test our implementation, and then use the arena class to play against other agents

In [13]:
rule = RuleSchieber()
game = GameSim(rule=rule)
agent = MyAgent()

np.random.seed(1)
game.init_from_cards(hands=deal_random_hand(), dealer=NORTH)

In [14]:
obs = game.get_observation()

In [15]:
cards = convert_one_hot_encoded_cards_to_str_encoded_list(obs.hand)
print(cards)
print(agent.action_trump(obs))
trump = agent.action_trump(obs)
assert trump == HEARTS

['DA', 'DK', 'D9', 'D6', 'HA', 'HQ', 'HJ', 'H8', 'H7']
1


In [16]:
# tell the simulation the selected trump
game.action_trump(trump)

In [17]:
# play the game to the end and print the result
while not game.is_done():
    game.action_play_card(agent.action_play_card(game.get_observation()))

print(game.state.points)

[ 10 147]


Another possibility to test agents locally is to use the arena. Let us play 100 games against the Random Agent and see if our trump methods makes any difference.


In [18]:
arena = Arena(nr_games_to_play=100)
arena.set_players(MyAgent(), AgentRandomSchieber(), MyAgent(), AgentRandomSchieber())

In [19]:
arena.play_all_games()

[........................................]  100/ 100 games played


In [20]:
print(arena.points_team_0.sum(), arena.points_team_1.sum())

8958.0 6742.0


Now you can continue with a rule based implemenation of the card play. Also look at the flask implementation of the service to see how you can get your agent online.